# Code of Video Streaming 

In [ ]:
import cv2
import numpy as np
import threading
import pygame
from tensorflow.keras.models import load_model


cnn_lstm_model = load_model('drowsiness_detection_cnn_lstm_model.h5')


categories = ['active', 'fatigue']  


pygame.mixer.init()
def play_alarm():
    if not pygame.mixer.music.get_busy(): 
        pygame.mixer.music.load("alert.wav")
        pygame.mixer.music.play()


def preprocess_frame(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  
    resized = cv2.resize(gray, (100, 100))  
    img_array = np.array(resized, dtype=np.float32) / 255.0  
    return img_array.reshape(1, 100, 100, 1)  


cap = cv2.VideoCapture(0)


fatigue_frames = 0
active_frames = 0
fatigue_threshold = 15  
active_threshold = 10   

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    img_array = preprocess_frame(frame)
    lstm_probs = cnn_lstm_model.predict(img_array, verbose=0)[0]  
    lstm_prediction = np.argmax(lstm_probs) 
    lstm_label = categories[lstm_prediction]  

 
    cv2.putText(frame, f"Prediction: {lstm_label}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

 
    if lstm_label == "fatigue" and lstm_probs[lstm_prediction] >= 0.50:
        fatigue_frames += 1
        active_frames = 0  
    else:
        active_frames += 1
        fatigue_frames = 0  

   
    if fatigue_frames >= fatigue_threshold:
        cv2.putText(frame, "DROWSINESS DETECTED!", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
        threading.Thread(target=play_alarm, daemon=True).start()

   
    if active_frames >= active_threshold:
        pygame.mixer.music.stop()

    cv2.imshow("Drowsiness Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()